In [17]:
import numpy as np
import matplotlib.pyplot as plt


class EigenvectorRLSolver:
    """
    Reinforcement Learning approach to find eigenvectors that satisfy (A-λI)x = 0
    """

    def __init__(
        self,
        matrix_A,
        lambda_val,
        learning_rate=0.01,
        max_iterations=10000,
        convergence_threshold=1e-10,
    ):
        """
        Initialize the solver with the matrix A and the eigenvalue lambda

        Args:
            matrix_A (numpy.ndarray): The matrix A in the equation
            lambda_val (float): The eigenvalue λ
            learning_rate (float): Learning rate for the gradient descent
            max_iterations (int): Maximum number of iterations
            convergence_threshold (float): Threshold for convergence
        """
        self.A = matrix_A
        self.lambda_val = lambda_val
        self.n = matrix_A.shape[0]  # Dimension of the matrix
        self.learning_rate = learning_rate
        self.max_iterations = max_iterations
        self.convergence_threshold = convergence_threshold

        # Initialize x randomly and normalize it
        self.x = np.random.rand(self.n)
        self.x = self.x / np.linalg.norm(self.x)

        # History for plotting convergence
        self.error_history = []

    def compute_reward(self, x):
        """
        Compute the negative error (reward) for the current solution
        The goal is to maximize this reward (or minimize the error)

        Args:
            x (numpy.ndarray): Current solution vector

        Returns:
            float: Negative error (reward)
        """
        # Calculate (A-λI)x
        error = np.linalg.norm(np.dot(self.A - self.lambda_val * np.eye(self.n), x))
        return -error

    def gradient_step(self):
        """
        Perform a gradient step to update the solution vector x
        """
        # Calculate (A-λI)
        A_minus_lambda_I = self.A - self.lambda_val * np.eye(self.n)

        # Calculate (A-λI)ᵀ(A-λI)x
        # derivative of squared error ||Ax - λx||² =  ||(A-λI)x||²
        gradient = np.dot(A_minus_lambda_I.T, np.dot(A_minus_lambda_I, self.x))

        # Update x in the direction that reduces the error
        self.x = self.x - self.learning_rate * gradient

        # Normalize x to prevent it from growing too large or becoming 0
        self.x = self.x / np.linalg.norm(self.x)

    def train(self):
        """
        Train the RL agent to find the eigenvector

        Returns:
            numpy.ndarray: The found eigenvector
            list: Error history for monitoring convergence
        """
        for i in range(self.max_iterations):
            # Current error
            error = -self.compute_reward(self.x)
            self.error_history.append(error)

            # Check for convergence
            if error < self.convergence_threshold:
                print(f"Converged after {i+1} iterations with error {error}")
                break

            # Perform a gradient step
            self.gradient_step()

            # Print progress occasionally
            if (i + 1) % 1000 == 0:
                print(f"Iteration {i+1}, Error: {error}")

        if i == self.max_iterations - 1:
            print(
                f"Reached maximum iterations ({self.max_iterations}) with error {error}"
            )

        return self.x, self.error_history

    def verify_solution(self):
        """
        Verify if the found solution is correct

        Returns:
            float: The verification error
        """
        # Calculate (A-λI)x
        result = np.dot(self.A - self.lambda_val * np.eye(self.n), self.x)

        # Calculate the error
        error = np.linalg.norm(result)

        print(f"Verification error: {error}")
        print(f"Solution vector x: {self.x}")

        # Also verify if it's an eigenvector of A with eigenvalue λ
        Ax = np.dot(self.A, self.x)
        lambda_x = self.lambda_val * self.x
        eigenvector_error = np.linalg.norm(Ax - lambda_x)

        print(f"Eigenvector verification error: {eigenvector_error}\n")

        return error

    def plot_convergence(self):
        """
        Plot the convergence of the error over iterations
        """
        plt.figure(figsize=(10, 5))
        plt.plot(range(len(self.error_history)), self.error_history)
        plt.xlabel("Iterations")
        plt.ylabel("Error")
        plt.title("Convergence of RL Eigenvector Solver")
        plt.yscale("log")  # Log scale to better see the convergence
        plt.grid(True)
        plt.show()

In [18]:

A = np.array([[4, 2], 
                [1, 3]])

# Eigenvalues: 5 and 2
# For eigenvalue 5, eigenvector is approximately [2, 1]
# For eigenvalue 2, eigenvector is approximately [-1, 1]

# Solve for eigenvalue 5
lambda_val = 5
solver = EigenvectorRLSolver(A, lambda_val, learning_rate=0.01, max_iterations=10000)
x, _ = solver.train()
solver.verify_solution()
# solver.plot_convergence()

# Solve for eigenvalue 2
lambda_val = 2
solver = EigenvectorRLSolver(A, lambda_val, learning_rate=0.01, max_iterations=10000)
x, _ = solver.train()
solver.verify_solution()
# solver.plot_convergence()

# Example 2: Try with a larger matrix
n = 4
B = np.random.rand(n, n)
B = (B + B.T) / 2  # Make it symmetric

# Calculate eigenvalues
eigenvalues, _ = np.linalg.eig(B)
print(f"Eigenvalues of matrix B: {eigenvalues}")

# Solve for the first eigenvalue
lambda_val = eigenvalues[0]
solver = EigenvectorRLSolver(B, lambda_val, learning_rate=0.01, max_iterations=20000)
x, _ = solver.train()
solver.verify_solution()
# solver.plot_convergence()

Converged after 220 iterations with error 9.176012851662652e-11
Verification error: 9.176012851662652e-11
Solution vector x: [0.89442719 0.4472136 ]
Eigenvector verification error: 9.176059954651336e-11

Converged after 254 iterations with error 9.173410552396258e-11
Verification error: 9.173410552396258e-11
Solution vector x: [ 0.70710678 -0.70710678]
Eigenvector verification error: 9.17342048255437e-11

Eigenvalues of matrix B: [ 1.54204243 -0.43713485 -0.15584551  0.61723642]
Iteration 1000, Error: 1.314636126818697e-05
Iteration 2000, Error: 2.4460015302226832e-09
Converged after 2373 iterations with error 9.932010694178382e-11
Verification error: 9.932010694178382e-11
Solution vector x: [0.424917   0.4397452  0.48919266 0.62190051]
Eigenvector verification error: 9.932014587995784e-11



9.932010694178382e-11

In [19]:
C = np.array([[6, 2, 1], 
              [2, 3, 1], 
              [1, 1, 2]])

eigenvalues, _ = np.linalg.eig(C)
print(f"Eigenvalues of matrix C: {eigenvalues}")

for lambda_val in eigenvalues:
    solver = EigenvectorRLSolver(C, lambda_val, learning_rate=0.01, max_iterations=15000)
    x, _ = solver.train()
    solver.verify_solution()

Eigenvalues of matrix C: [7.33949226 2.29511673 1.36539102]
Converged after 84 iterations with error 9.05115257935239e-11
Verification error: 9.05115257935239e-11
Solution vector x: [0.85772795 0.45183049 0.24525898]
Eigenvector verification error: 9.05118278072956e-11

Iteration 1000, Error: 0.00016044384414956958
Iteration 2000, Error: 2.7227437979530407e-08
Converged after 2646 iterations with error 9.985849927560143e-11
Verification error: 9.985849927560143e-11
Solution vector x: [-0.50752384  0.6681011   0.54411439]
Eigenvector verification error: 9.98583970748016e-11

Iteration 1000, Error: 0.0001437840196109882
Iteration 2000, Error: 2.4400252518857792e-08
Converged after 2634 iterations with error 9.931427568074987e-11
Verification error: 9.931427568074987e-11
Solution vector x: [ 0.08198967 -0.59117691  0.80236373]
Eigenvector verification error: 9.931436522085677e-11



In [20]:
n = 5
D = np.random.rand(n, n)
D = (D + D.T) / 2  # Make it symmetric

eigenvalues, _ = np.linalg.eig(D)
print(f"Eigenvalues of matrix D: {eigenvalues}")

for lambda_val in eigenvalues[:3]:  # Test first 3 eigenvalues
    solver = EigenvectorRLSolver(D, lambda_val, learning_rate=0.01, max_iterations=20000)
    x, _ = solver.train()
    solver.verify_solution()


Eigenvalues of matrix D: [ 2.82928544  0.76781365 -0.25660604  0.19054269 -0.02773246]
Converged after 516 iterations with error 9.866013846969059e-11
Verification error: 9.866013846969059e-11
Solution vector x: [0.53004047 0.37821605 0.45430047 0.43883437 0.42076739]
Eigenvector verification error: 9.86603638724101e-11

Iteration 1000, Error: 0.014610067232350235
Iteration 2000, Error: 0.0005183827348946975
Iteration 3000, Error: 1.8406965562361348e-05
Iteration 4000, Error: 6.536042115291293e-07
Iteration 5000, Error: 2.3208522403789403e-08
Iteration 6000, Error: 8.24100498723232e-10
Converged after 6632 iterations with error 9.995185533432954e-11
Verification error: 9.995185533432954e-11
Solution vector x: [ 0.30331153 -0.40570909  0.58963096 -0.0243715  -0.6286045 ]
Eigenvector verification error: 9.99518080983018e-11

Iteration 1000, Error: 0.09997736139631537
Iteration 2000, Error: 0.053161531383135605
Iteration 3000, Error: 0.031719737859124485
Iteration 4000, Error: 0.018891533

In [21]:
E = np.diag([10, 5, -2, 3])

eigenvalues, _ = np.linalg.eig(E)
print(f"Eigenvalues of matrix E: {eigenvalues}")

for lambda_val in eigenvalues:
    solver = EigenvectorRLSolver(E, lambda_val, learning_rate=0.01, max_iterations=5000)
    x, _ = solver.train()
    solver.verify_solution()
# diagonal matrixx

Eigenvalues of matrix E: [10.  5. -2.  3.]
Converged after 82 iterations with error 8.905763972828738e-11
Verification error: 8.905763972828738e-11
Solution vector x: [ 1.00000000e+00  1.78115279e-11 -9.82191034e-30  9.43884405e-25]
Eigenvector verification error: 8.905763972828738e-11

Converged after 601 iterations with error 9.965855483647343e-11
Verification error: 9.965855483647343e-11
Solution vector x: [1.62852807e-075 1.00000000e+000 7.77126717e-176 4.98292774e-011]
Eigenvector verification error: 9.965855483647342e-11

Converged after 89 iterations with error 8.096920368689948e-11
Verification error: 8.096920368689948e-11
Solution vector x: [1.12473628e-33 2.23465749e-26 1.00000000e+00 1.61938407e-11]
Eigenvector verification error: 8.096920368689948e-11

Converged after 609 iterations with error 9.733577929547412e-11
Verification error: 9.733577929547412e-11
Solution vector x: [1.49567163e-178 4.86678896e-011 2.89519663e-076 1.00000000e+000]
Eigenvector verification error: 9.

In [22]:
F = np.array([[1, 0.99], 
              [0.99, 1]])

eigenvalues, _ = np.linalg.eig(F)
print(f"Eigenvalues of matrix F: {eigenvalues}")

for lambda_val in eigenvalues:
    solver = EigenvectorRLSolver(F, lambda_val, learning_rate=0.01, max_iterations=20000)
    x, _ = solver.train()
    solver.verify_solution()
# singular matrix

Eigenvalues of matrix F: [1.99 0.01]
Converged after 579 iterations with error 9.709154756884338e-11
Verification error: 9.709154756884338e-11
Solution vector x: [0.70710678 0.70710678]
Eigenvector verification error: 9.70915344694601e-11

Converged after 637 iterations with error 9.746625219675208e-11
Verification error: 9.746625219675208e-11
Solution vector x: [-0.70710678  0.70710678]
Eigenvector verification error: 9.746625359416695e-11



In [23]:
n = 10
G = np.random.rand(n, n)
G = (G + G.T) / 2  # Make it symmetric

eigenvalues, _ = np.linalg.eig(G)
print(f"Eigenvalues of matrix G: {eigenvalues}")

for lambda_val in eigenvalues[:5]:  # Test first 5 eigenvalues
    solver = EigenvectorRLSolver(G, lambda_val, learning_rate=0.01, max_iterations=30000)
    x, _ = solver.train()
    solver.verify_solution()
# large singular matrix

Eigenvalues of matrix G: [ 4.9172304  -0.53244645 -0.50044075 -0.35598003 -0.05492927  1.03888772
  0.28813837  0.53066797  0.69611255  0.74194875]
Converged after 140 iterations with error 9.981835352073189e-11
Verification error: 9.981835352073189e-11
Solution vector x: [0.3612889  0.34634895 0.30771013 0.32882766 0.25478231 0.25378434
 0.34911252 0.27595128 0.31257782 0.34877716]
Eigenvector verification error: 9.981841035599633e-11

Iteration 1000, Error: 0.04763934990261294
Iteration 2000, Error: 0.03888533407384891
Iteration 3000, Error: 0.03470995449246063
Iteration 4000, Error: 0.032160847861604204
Iteration 5000, Error: 0.030644199455546972
Iteration 6000, Error: 0.02975504514220157
Iteration 7000, Error: 0.029231334604360934
Iteration 8000, Error: 0.028914591440219777
Iteration 9000, Error: 0.028712998920219307
Iteration 10000, Error: 0.028574697653462145
Iteration 11000, Error: 0.028470820663016552
Iteration 12000, Error: 0.028385430039245256
Iteration 13000, Error: 0.028309

In [24]:
import numpy as np

# Create a random 10x10 matrix
H = np.random.rand(10, 10)

# Make it symmetric
H = (H + H.T) / 2

# Make it singular by setting the last row and column to be a linear combination of others
H[:, -1] = H[:, -2]  # Copy the second last column to the last column
H[-1, :] = H[-2, :]  # Copy the second last row to the last row

# Compute eigenvalues
eigenvalues, _ = np.linalg.eig(H)
print(f"Eigenvalues of matrix H: {eigenvalues}")

# Train on the first 7 eigenvalues
for lambda_val in eigenvalues[:7]:
    solver = EigenvectorRLSolver(H, lambda_val, learning_rate=0.01, max_iterations=30000)
    x, _ = solver.train()
    solver.verify_solution()

# Test on the remaining eigenvalues
print("\nTesting on remaining eigenvalues...\n")
for lambda_val in eigenvalues[7:]:
    solver = EigenvectorRLSolver(H, lambda_val, learning_rate=0.01, max_iterations=30000)
    x, _ = solver.train()
    solver.verify_solution()


Eigenvalues of matrix H: [ 4.75288490e+00  9.42310688e-01  6.25470634e-01 -8.71186434e-01
 -8.24127861e-01 -5.99940432e-01  2.02803362e-01 -2.94056518e-01
 -2.20595702e-01  3.67669240e-33]
Converged after 144 iterations with error 9.024323508088503e-11
Verification error: 9.024323508088503e-11
Solution vector x: [0.26016018 0.30431561 0.29449695 0.28502823 0.25834223 0.39954118
 0.32477961 0.33942533 0.33516707 0.33516707]
Eigenvector verification error: 9.024330333084465e-11

Iteration 1000, Error: 0.04917599533361738
Iteration 2000, Error: 0.018197758584843803
Iteration 3000, Error: 0.006674859414382952
Iteration 4000, Error: 0.002445280633934529
Iteration 5000, Error: 0.0008956596966641806
Iteration 6000, Error: 0.0003280557536683673
Iteration 7000, Error: 0.00012015752807404496
Iteration 8000, Error: 4.401028053774623e-05
Iteration 9000, Error: 1.6119711512431197e-05
Iteration 10000, Error: 5.904190888572095e-06
Iteration 11000, Error: 2.1625368415055836e-06
Iteration 12000, Error: